In [2]:
import numpy as np
# import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt

from math import sqrt
from matplotlib import animation, rc
# from numba import jit, guvectorize, prange
# from IPython.display import HTML

rc('animation', html='jshtml')

In [3]:
class MembraneDynamics:
    
    def __init__(self,
                 surface_area,
                 bcr_number,
                 bcr_radius,
                 bcr_rafts_radius,
                 bcr_diffusion_coef,
                 free_rafts_number,
                 free_rafts_radius,
                 raft_diffusion_coef,
                ):
        
        self.side_length = np.sqrt(surface_area)
        
        # BCR
        self.bcr_coordinates = np.random.rand(bcr_number, 2) * self.side_length
        self.bcr_radius = bcr_radius    # ????
        self.cluster_number_bcr = np.arange(bcr_number)
#         self.bcr_exist = np.ones(bcr_number)
        self.bcr_diffusion_coef = bcr_diffusion_coef
        
        # Lipid rafts
        rafts_number = bcr_number + free_rafts_number
        self.lipid_rafts_coordinates = np.empty((rafts_number, 2))
        self.lipid_rafts_coordinates[:bcr_number] = self.bcr_coordinates
        self.lipid_rafts_coordinates[bcr_number:] = np.random.rand(free_rafts_number, 2) * self.side_length
        self.lipid_rafts_radii = np.empty((rafts_number, 1))
        self.lipid_rafts_radii[:bcr_number] = bcr_rafts_radius
        self.lipid_rafts_radii[bcr_number:] = free_rafts_radius
        self.cluster_number_lr = np.arange(rafts_number)
        self.raft_diffusion_coef = raft_diffusion_coef
    
    
    def evolve_system(self, dt): # пока работает только для рафтов
        self.diffusion(self.cluster_number_lr,
                       self.lipid_rafts_coordinates,
                       self.lipid_rafts_radii,
                       self.raft_diffusion_coef,
                       sqrt(dt))
        self.fusion(self.cluster_number_lr,
                    self.lipid_rafts_coordinates,
                    self.lipid_rafts_radii)
    
    
    def get_rafts(self):
        return (self.cluster_number_lr,
                self.lipid_rafts_coordinates,
                self.lipid_rafts_radii)
    
    
    @staticmethod
    def distance_matrix(points):
        x = points[:, 0:1]
        y = points[:, 1:2]
        return np.sqrt((x - x.T)**2 + (y - y.T)**2)
    
    
    @staticmethod
    def combined_params(coordinates, radii):
        radii_sq_sum = (radii**2).sum()
        new_center = (coordinates * radii**2).sum(axis=0) / radii_sq_sum
        return new_center, sqrt(radii_sq_sum)
    
    
    @staticmethod
    def diffusion(clusters, centers, radii, k, time_step_root):
        real_index = clusters[clusters >= 0]

        shift = np.random.normal(size=(real_index.size, 2)) * k * time_step_root    
        centers[real_index] += shift / radii[real_index] # это плохая нормировка, но я исправлю

        # Тороидальные граничные условия
        centers[centers < 0] += side_length
        centers[centers > side_length] -= side_length
    
    
    @staticmethod
    def fusion(clusters, centers, radii):
        real_index = clusters[clusters >= 0]
        
        n_clusters = real_index.size
        dist_matr = MembraneDynamics.distance_matrix(centers[real_index])
        clusters_radii = radii[real_index].reshape(-1,1)
        overlap_mask = clusters_radii + clusters_radii.T
        
        fusion_list = []
        for i in range(n_clusters):
            for j in range(i+1, n_clusters):
                if dist_matr[i,j] <= overlap_mask[i,j]:
                    fusion_list.append((i,j))
                    
        connectivity_comps = MembraneDynamics._connectivity_components(n_clusters,
                                                                       fusion_list)        
        for component in connectivity_comps:
            
            new_center, new_radius = MembraneDynamics.combined_params(
                centers[real_index[component]],
                radii[real_index[component]],
            )
            
            dominant_class = component[0]
            centers[real_index[dominant_class]] = new_center
            radii[real_index[dominant_class]] = new_radius
            for i in component[1:]:
                clusters[real_index[i]] = -1
                    
            
    @staticmethod
    def _connectivity_components(n_elements, edges):
        '''
        
        '''
        # breadth-first search
        def bfs(i, vertex, adjacency_list, comps):
            if comps[vertex]:
                return False

            comps[vertex] = i
            for v in adjacency_list[vertex]:
                bfs(i, v, adjacency_list, comps)

            return True
        
        # creating adjacency list (dict) from edges
        adjacency_list = {}
        for a, b in edges:
            adjacency_list[a] = adjacency_list.get(a, []) + [b]
            adjacency_list[b] = adjacency_list.get(b, []) + [a]
        
        # searching for connectivity components
        elements = np.arange(n_elements)
        comps = np.zeros_like(elements)
        i = 1
        for vertex in adjacency_list.keys():
            if bfs(i, vertex, adjacency_list, comps):
                i += 1
        
        # extracting components
        connectivity_comps = []
        for k in range(1, i):
            connectivity_comps.append(elements[comps == k])

        return connectivity_comps

In [320]:
# # Тест работы алгоритма в режиме отладки
# np.random.seed(0)

# membrane_lr_dynamics = MembraneDynamics(
#     surface_area=SURFACE_AREA,
#     bcr_number=BCR_NUMBER,
#     bcr_radius=BCR_RADIUS,
#     bcr_rafts_radius=BCR_RAFTS_RADIUS,
#     bcr_diffusion_coef=BCR_DIFFUSION_COEFFICIENT,
#     free_rafts_number=FREE_RAFTS_NUMBER,
#     free_rafts_radius=FREE_RAFTS_RADIUS,
#     raft_diffusion_coef=RAFT_DIFFUSION_COEFFICIENT,
# )

# for i in range(130):
#     print(f'\t{i}')
#     membrane_lr_dynamics.evolve_system(i*time_step)

In [319]:
# Тест функции поиска компонент связности

# import igraph

# g = igraph.Graph.GRG(100, 0.08)
# a = list(map(np.asarray, filter(lambda s: len(s) > 1, g.components())))
# b = MembraneDynamics._connectivity_components(100, g.get_edgelist())
# a, b

In [10]:
# Параметры модели

# Modeling area
SURFACE_AREA = 2

# Time
SIMULATION_TIME = 1
TIME_STEPS = 10

# BCR
BCR_NUMBER = 60
BCR_RADIUS = 0.005
BCR_RAFTS_RADIUS = 0.01
BCR_DIFFUSION_COEFFICIENT = 0.000005

# Lipid rafts
FREE_RAFTS_NUMBER = 40
FREE_RAFTS_RADIUS = 0.02
RAFT_DIFFUSION_COEFFICIENT = 0.0001

##############

side_length = np.sqrt(SURFACE_AREA)
time_step = SIMULATION_TIME / TIME_STEPS

In [11]:
# Создание анимации работающей модели

np.random.seed(1)

membrane_lr_dynamics = MembraneDynamics(
    
    # Modeling area
    surface_area=SURFACE_AREA,
    
    # BCR
    bcr_number=BCR_NUMBER,
    bcr_radius=BCR_RADIUS,
    bcr_rafts_radius=BCR_RAFTS_RADIUS,
    bcr_diffusion_coef=BCR_DIFFUSION_COEFFICIENT,
    
    # Lipid rafts
    free_rafts_number=FREE_RAFTS_NUMBER,
    free_rafts_radius=FREE_RAFTS_RADIUS,
    raft_diffusion_coef=RAFT_DIFFUSION_COEFFICIENT,
)

# set up figure and animation
fig, ax = plt.subplots(1, figsize=(10,10))
ax.set_xlim(0, side_length)
ax.set_ylim(0, side_length)

real_index, lipid_rafts_centers, lipid_rafts_radii = membrane_lr_dynamics.get_rafts()
drawing_rafts = np.empty(lipid_rafts_radii.size, dtype=np.object_)
for i in range(lipid_rafts_radii.size):
    drawing_rafts[i] = plt.Circle(
        xy=lipid_rafts_centers[i],
        radius=lipid_rafts_radii[i],
        alpha=0.3,
    )

for raft in drawing_rafts:
    ax.add_patch(raft)

    
def init():
    """initialize animation"""
    global drawing_rafts
    return drawing_rafts

def animate(i):
    """perform animation step"""
    global drawing_rafts, membrane_lr_dynamics
    
    membrane_lr_dynamics.evolve_system(time_step)
    real_index, lipid_rafts_centers, lipid_rafts_radii = membrane_lr_dynamics.get_rafts()
    
    i = 0
    for raft in drawing_rafts:
        if real_index[i] >= 0:
            raft.set_center(lipid_rafts_centers[i])
            raft.set_radius(lipid_rafts_radii[i])
        else:
            raft.set_alpha(0)
        i += 1
        
    return drawing_rafts

ani = animation.FuncAnimation(fig, animate, frames=500,
                              interval=30, blit=True, init_func=init)

plt.close()

In [ ]:
# Анимация воспроизводится в интерактивном режиме
ani